# Justification Simple Diabetes Metabolism Model (Insulin and Carb Only)

**Authors:** Ed Nykaza<br>
**External Reviewers and Special Thanks To:** Lane Desborough<br>
**Last Modified:** April 10, 2020<br>
**Audience:** External reviewers, internal review, FDA Q-sub<br>
**Current Status:** Internal Review before FDA Q-sub

## Description

This notebook contains the details of the the insulin model and a carb model that are used as Tidepool's simple diabetes metabolism model for the purposes of simulating the Loop Algorithm. 

### Insulin Model
The **insulin model** used is a biexponential insulin model (Palerm, 2009), which has been shown to be equivalent to the Hovorka Model (Bondia et al., 2018):

$I(t) = \frac{U_0}{K_{cl}(\tau_2-\tau_1)} (e^{\frac{-t}{\tau_2}}-e^{\frac{-t}{\tau_1}})$, 

where:

* $I(t)=$ plasma insulin (U/L) at time $t$
* $U_0=$ amount of insulin (U) delivered at time $t=0$
* $K_{cl}=$ insulin clearance rate (nominally 1 L/min)
* $\tau_1=$ time constant of 55 min
* $\tau_2=$ time constant of 70 min

NOTE: The above parameters assume body weight of 70 kg (~154 lbs) 

Here are the references:
* Bondia, Jorge, et al. "Insulin estimation and prediction: a review of the estimation and prediction of subcutaneous insulin pharmacokinetics in closed-loop glucose control." IEEE Control Systems Magazine 38.1 (2018): 47-66).
* Palerm, Cesar C. "Physiologic insulin delivery with insulin feedback: a control systems perspective." IFAC Proceedings Volumes 42.12 (2009): 31-36.

### Carb Model
The **carb model** is a derivative of the general model given in Percival et al. 2011 (Eq. 2) and Cescon 2013 (Eq. 8.4):

$G(s)=e^{-s\theta} \frac{KD}{s(s\tau + 1)}$,

which after taking the Laplace Transform can be transformed into this time domain equation:

$G(t)=KD(1-e^{\frac{\theta-t}{\tau}})H(t-\theta)$

where:

* $G=$ blood glucose
* $s=$ Laplace state variable
* $\theta=$ delay (AKA dead time)
* $K=$ gain (ISF / CIR) (mg/dL/g)
* $D=$ amount of carbs delivered (g)
* $\tau=$ time constant
* $H=$ Heaviside Function

Here are the references:
* Cescon, Marzia. Modeling and prediction in diabetes physiology. Department of Automatic Control, Lund University, 2013.
* Percival, M. W., et al. "Development of a multi-parametric model predictive control algorithm for insulin delivery in type 1 diabetes mellitus using clinical parameters." Journal of process control 21.3 (2011): 391-404.


## Instructions

1.   Please run this in Playground Mode (click File, and Open in playground mode) or create a copy so you don't overwrite this example (i.e., File, "Save a copy in Drive..."
2.   Before you run the code be sure to select "Connect" in the top right-hand corner of the webpage.
3.   To run the code, you press shift-return to execute each cell, or you can press the Run button (above), or you can click on the play button on each cell.
4.   You will be prompted to connect your collaboratory instance to google drive, which is required to pull the SEG data into the notebook. Follow the link and copy/paste the auth string back into this notebook.

# Insulin Model

In [0]:
# LOAD LIBRARIES, FUNCTIONS, AND DATA
import numpy as np
import plotly.express as px

In [0]:
# Insulin Component of the Simple Diabets Metabolism Model
''' We use the biexponential model given in Palerm (2009), which has been
shown to be equivalent to the Hovorka Model(Bondia et al., 2018).

I(t) = U0 * (1 / (Kcl * (tau2 - tau1))) * (e^(-t/tau2) - e^(-t/tau1))
where:
    I(t) = plasma insulin (U/L)
    t = time (min)
    U0 = amount of insulin delivered by pump (U)
    Kcl = insulin clearance rate (nominally 1 L/min)
    tau1 = 55 min
    tau2 = 70 min
    NOTE: Parameters above assume body weight of 70 kg (~154 lbs) 
'''
tau1 = 55 
tau2 = 70 
Kcl = 1
U0 = 1
t = np.arange(0, 8*60, 1)  # in minutes

I = U0 * (1 / (Kcl * (tau2 - tau1))) * (np.exp(-t/tau2) - np.exp(-t/tau1))
fig = px.line(
    title="Insulin Activity Curve",
    x=t,
    y=I,
    labels={"x":"time (min)", "y":"Insulin (U/L)"}
)
fig.show()


In [0]:
# insulin absorbed
''' the amount of insulin absorbed over the insulin duration
'''
ia = np.cumsum(I)
fig = px.line(
    title="Percent of Insulin Absorbed Over Time",
    x=t,
    y=ia, 
    labels={"x":"time (min)", "y":"Insulin Absorbed (U)"}
)
fig.show()

In [0]:
# insulin-on-board (IOB)
''' the amount of insulin on board will be the amount delivered at t=0
minus the total amount absorbed over the insulin duration
'''
iob = U0 - ia
fig = px.line(
    title="Insulin On Board",
    x=t, 
    y=iob, 
    labels={"x":"time (min)", "y":"IOB (U)"}
)
fig.show()


In [0]:
# cumulative effect of insulin on blood glucose
''' the cumulative effect of the insulin dose will be the
total amount of insulin absorbed times the insulin sensitivity  
'''
ISF = 50  # insulin sensitivity factor (ISF) mg/dL/U
insulin_effect = -ISF * ia
fig = px.line(
    title="Cumulative Effect of 1U of Insulin and an ISF of 50 mg/dL/U",
    x=t,
    y=insulin_effect, 
    labels={"x":"time (min)", "y":"Cumulative Drop in BG (mg/dL)"}
)
fig.show()


In [0]:
# Expected Change in BG every 5 minutes
''' lastly we can estimate the expected change in BG
every 5 minutes by taking the derivative (rate of change)
of the cumulative insulin effect
'''
t_5min = np.arange(0, 8*60, 5)
ie_5min = insulin_effect[t_5min]
expected_drop = np.append(0, ie_5min[1:] - ie_5min[:-1])
fig = px.scatter(x=t_5min, y=expected_drop, labels={"x":"Time (min)", "y":"Change in BG (mg/dL)"}, title="Expected Drop in BG Every 5 Minutes")
fig.show()

# Carb Model

Using data from Table 8.1 in Cescon (2013) for parameters $\tau$ and $\theta$:

*   median $\tau \approx$ 42 minutes
*   median $\theta \approx$ 20 minutes



In [0]:
tau = 42
theta = 20

ISF = 50
CIR = 12.5
K = ISF / CIR 

time_of_carbs = 0
carb_amount = 25

t = np.arange(0, 480, 1)  # minutes
delta_t = t[1] - t[0]  # minutes
carb_index = np.int(np.ceil((time_of_carbs)/delta_t))

c_t = K*carb_amount*(1-np.exp((theta-t)/tau))*np.heaviside(t-theta, 1)

fig = px.line(x=t, y=c_t, labels={"x":"Time (min)", "y":"Change in BG (mg/dL)"}, title="Cumulative Effect of Carbs")
fig.add_scatter(x=[time_of_carbs], y=[c_t[carb_index]], name="Carbs Consumed")
fig.show()

In [0]:
# Expected Change in BG every 5 minutes
''' we can estimate the expected change in BG
every 5 minutes by taking the derivative (rate of change)
of the cumulative carb effect
'''
t_5min = np.arange(0, 8*60, 5)
ce_5min = c_t[t_5min]
expected_increase = np.append(0, ce_5min[1:] - ce_5min[:-1])
fig = px.scatter(x=t_5min, y=expected_increase, labels={"x":"Time (min)", "y":"Change in BG (mg/dL)"}, title="Expected Drop in BG Every 5 Minutes")
fig.show()
